In [ ]:
import pandas as pd
import requests
import time

# ✅ 카카오 REST API 키 입력
KAKAO_API_KEY = ''

# ✅ 주소 → 좌표 변환 함수
def get_coordinates(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        #200은 성공 코드값
        return None, None

    result = response.json()
    documents = result.get("documents", [])
    if documents:
        x = float(documents[0]["x"])  # 경도
        y = float(documents[0]["y"])  # 위도
        return y, x
    return None, None

# ✅ 주소 목록 CSV 파일 불러오기
df = pd.read_csv("input.csv")  # 주소가 들어 있는 열 이름은 '주소'라고 가정

# ✅ 위도, 경도 컬럼 추가
lat_list = []
lon_list = []

for address in df['주소']:
    lat, lon = get_coordinates(address)
    lat_list.append(lat)
    lon_list.append(lon)
    time.sleep(0.2)  # 과도한 요청 방지를 위해 0.2초 지연

df['위도'] = lat_list
df['경도'] = lon_list

# ✅ 결과 저장
df.to_csv("Data_saramin_with_coordinates.csv", index=False, encoding='utf-8-sig')
print("완료: Data_saramin_with_coordinates.csv 저장됨")


In [ ]:
from folium.plugins import MarkerCluster
import pandas as pd
import folium

# 데이터 로딩
df = pd.read_csv("../data/Data_saramin_with_coordinates.csv", encoding='utf-8')
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
df_valid = df.dropna(subset=['위도', '경도'])

# 지도 초기화
map_center = df_valid[['위도', '경도']].mean().tolist()
m = folium.Map(location=map_center, zoom_start=11)

# 클러스터 생성
marker_cluster = MarkerCluster().add_to(m)

# 마커 추가 (클러스터에)
for idx, row in df_valid.iterrows():
    lat, lon = row['위도'], row['경도']
    address = row.get('주소', '주소 없음')

    folium.Marker(
        [lat, lon]
        #popup=f"Index: {idx}<br>주소: {address}",
        #tooltip=address
    ).add_to(marker_cluster)

# 저장
m.save("company_address_map.html")
print("✅ 클러스터 지도 저장 완료: company_address_map.html")


✅ 클러스터 지도 저장 완료: company_clustered_map.html
